In [6]:
#!/usr/bin/python
import sys
import math
import glob
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import numpy.linalg as lin

# Loading convenient environment
# %pylab
%matplotlib inline

plt.rcParams['figure.figsize']=[20.,10.]

In [30]:
# bloc='/Users/vassaux/picasso_home'
bloc='/home/maxime'
floc='/muc_work/local_projects/DeaLAMMPS/build_init/macroscale_state/out_bidisp/init.stiff'
sloc=bloc+floc

In [31]:
# Setting dimension number of the mechanical problem
dim = 3

In [32]:
# Reading saved stiffness file from DeaLAMMPS
x = []
file_in = open(sloc, 'r')
for y in file_in.readlines():
    x.append(float(y))

In [33]:
# Loading the stiffness coefficients in the fourth order stiffness tensor 'C'
i=0
stf4 = np.zeros((dim,dim,dim,dim))
for k in range(0, dim):
    for l in range(k, dim):
        for m in range(0, dim):
            for n in range(m, dim):
                stf4[k][l][m][n] = x[i]
                stf4[k][l][n][m] = stf4[k][l][m][n]
                stf4[l][k][m][n] = stf4[k][l][m][n]
                stf4[l][k][n][m] = stf4[k][l][m][n]
#                 print(stf4[k][l][m][n])
                i = i+1

In [34]:
# Converting the fourth order stiffness tensor into the 2nd order stiffness tensor based on Voigt notation
stf2 = np.zeros((2*dim,2*dim))

for i in range(0, 2*dim):
    if     (i==(3+0)):
        k=0 
        l=1
    elif(i==(3+1)):
        k=0 
        l=2
    elif(i==(3+2)):
        k=1 
        l=2
    else:
        k=i
        l=i

    for j in range(0, 2*dim):
        if     (j==(3+0)):
            m=0
            n=1
        elif(j==(3+1)):
            m=0
            n=2
        elif(j==(3+2)):
            m=1
            n=2
        else :
            m=j
            n=j
        factor = 1.0
        if i>dim-1:
            factor *= math.sqrt(2)
        if j>dim-1:
            factor *= math.sqrt(2)
            
        stf2[i][j]=factor*stf4[k][l][m][n]
#         if((i>dim-1 or j>dim-1) and i!=j):
#             stf2[i][j]=0.0

In [35]:
dstf2=pd.DataFrame(stf2)
dstf2

,0,1,2,3,4,5
0,3.485870e+09,2.296129e+09,2.286283e+09,-1.536518e+07,-8.122842e+07,-3.245116e+07
1,2.296129e+09,3.468453e+09,2.419613e+09,-1.472833e+08,-1.057042e+08,2.516130e+07
2,2.286283e+09,2.419613e+09,3.402209e+09,-1.352985e+08,-1.872006e+07,-1.233832e+08
3,-1.536518e+07,-1.472833e+08,-1.352985e+08,1.014982e+09,-2.546064e+07,1.596220e+06
4,-8.122842e+07,-1.057042e+08,-1.872006e+07,-2.546064e+07,1.206307e+09,-8.838382e+07
5,-3.245116e+07,2.516130e+07,-1.233832e+08,1.596220e+06,-8.838382e+07,1.004022e+09


In [36]:
# Compute the compliance matrix
com2=lin.inv(stf2)
dcom2=pd.DataFrame(com2)
#dcom2

In [37]:
eta_nu=np.zeros((2*dim,2*dim))
mod=np.zeros(2*dim)

In [38]:
# Compute the modulus (young's (0,1,2) and shear (3,4,5))
for i in range(0, 2*dim):
    mod[i]=1.0/com2[i][i]

In [39]:
dmod=pd.DataFrame(mod)
dmod

,0
0,1.682736e+09
1,1.499074e+09
2,1.455194e+09
3,1.002148e+09
4,1.193002e+09
5,9.864861e+08


In [40]:
for i in range(0, 2*dim):
    for j in range(0, 2*dim):
        if(i!=j):
            eta_nu[i][j]=-1.0*com2[i][j]*mod[i]

In [41]:
deta_nu=pd.DataFrame(eta_nu)
deta_nu

,0,1,2,3,4,5
0,0.000000,0.384538,0.402293,0.093652,-0.025046,0.005126
1,0.342568,0.000000,0.480916,-0.077266,-0.052076,0.090771
2,0.347894,0.466839,0.000000,-0.059131,0.038787,-0.119835
3,0.055774,-0.051653,-0.040722,0.000000,-0.022678,-0.002314
4,-0.017756,-0.041444,0.031798,-0.026997,0.000000,-0.083614
5,0.003005,0.059733,-0.081237,-0.002277,-0.069140,0.000000


In [19]:
# Verification of reciprocity conditions
rcon=np.zeros((2*dim,2*dim))
for i in range(0, 2*dim):
    for j in range(0, 2*dim):
#         print(i, j,  mod[i]*eta_nu[i][j] - mod[j]*eta_nu[j][i], mod[i]*eta_nu[i][j], mod[j]*eta_nu[j][i])
        rcon[i][j] = mod[i]*eta_nu[j][i] - mod[j]*eta_nu[i][j]

In [20]:
drcon=pd.DataFrame(rcon)
drcon

,0,1,2,3,4,5
0,0.000000e+00,-1.192093e-07,1.192093e-07,-5.960464e-08,-1.490116e-08,0.000000e+00
1,1.192093e-07,0.000000e+00,0.000000e+00,2.980232e-08,2.980232e-08,0.000000e+00
2,-1.192093e-07,0.000000e+00,0.000000e+00,1.490116e-08,-2.048910e-08,-3.725290e-09
3,5.960464e-08,-2.980232e-08,-1.490116e-08,0.000000e+00,1.490116e-08,0.000000e+00
4,1.490116e-08,-2.980232e-08,2.048910e-08,-1.490116e-08,0.000000e+00,-2.980232e-08
5,0.000000e+00,0.000000e+00,3.725290e-09,0.000000e+00,2.980232e-08,0.000000e+00


### Conclusions

The homogenization issue related to mono-directional straining is not affected by bigger time-samples, or by modifiying the equilibration process (300K+5MPa).

The irregaluraties appear mostly on shear related terms (all the terms outside of (i,j)<dim). But the terms in (i,j)<dim seem to be also a little bit sensible.

The use of bidirectional homogenization levels perfectly and consistently these irregularities, which is surprising...
... and actually not, because bi-directional implies twice bigger strain perturbation.

Using 0.025 (instead of 0.005) as amplitude of strain perturbation, very decent results are obtained with mono-directional homogenization! With mono-directional, use at the very least 0.01 strain perturbation.


In [ ]:
# 300K

In [10]:
## Bidisplaced stiffness tensor

,0,1,2,3,4,5
0,3.485870e+09,2.296129e+09,2.286283e+09,-1.536518e+07,-8.122842e+07,-3.245116e+07
1,2.296129e+09,3.468453e+09,2.419613e+09,-1.472833e+08,-1.057042e+08,2.516130e+07
2,2.286283e+09,2.419613e+09,3.402209e+09,-1.352985e+08,-1.872006e+07,-1.233832e+08
3,-1.536518e+07,-1.472833e+08,-1.352985e+08,1.014982e+09,-2.546064e+07,1.596220e+06
4,-8.122842e+07,-1.057042e+08,-1.872006e+07,-2.546064e+07,1.206307e+09,-8.838382e+07
5,-3.245116e+07,2.516130e+07,-1.233832e+08,1.596220e+06,-8.838382e+07,1.004022e+09


In [39]:
## +1 monodisplaced stiffness tensor

,0,1,2,3,4,5
0,3.696171e+09,3.052853e+09,2.492602e+09,1.591550e+08,2.778299e+08,7.701664e+08
1,3.052853e+09,4.573311e+09,2.992247e+09,4.609242e+08,7.788666e+08,1.346555e+09
2,2.492602e+09,2.992247e+09,3.193475e+09,-3.385506e+08,5.994583e+07,4.563741e+08
3,1.591550e+08,4.609242e+08,-3.385506e+08,2.298516e+08,-4.750227e+08,2.938753e+08
4,2.778299e+08,7.788666e+08,5.994583e+07,-4.750227e+08,1.181468e+09,4.833730e+08
5,7.701664e+08,1.346555e+09,4.563741e+08,2.938753e+08,4.833730e+08,2.142238e+09


In [50]:
## -1 monodisplaced stiffness tensor

,0,1,2,3,4,5
0,3.275570e+09,1.539405e+09,2.079965e+09,-1.898856e+08,-4.402868e+08,-8.350688e+08
1,1.539405e+09,2.363595e+09,1.846979e+09,-7.554909e+08,-9.902752e+08,-1.296232e+09
2,2.079965e+09,1.846979e+09,3.610943e+09,6.795346e+07,-9.738577e+07,-7.031406e+08
3,-1.898856e+08,-7.554909e+08,6.795346e+07,1.800112e+09,4.241014e+08,-2.906830e+08
4,-4.402868e+08,-9.902752e+08,-9.738577e+07,4.241014e+08,1.231145e+09,-6.601407e+08
5,-8.350688e+08,-1.296232e+09,-7.031406e+08,-2.906830e+08,-6.601407e+08,-1.341937e+08
